# Capstone Project

# description of the problem

### There are lots of factors infect the housing price and in this project, I am gonna using the venue data to deplore the relationship between the venue type and the housing price in different philadelphia area. This project could be used for potential real estate investment analysis.

# A description of the data

### The data I am using in this project is: 1. the neighborhood list from wikipedia. 2. utilize the foursquare api to get the latitude and longitude of different neighborhoods and venues. 3. manually download the housing price for different neighborhood in philadelphia

In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /Users/daiyiping/anaconda3

  added / updated specs: 
    - geopy


The following NEW packages will be INSTALLED:

    conda-package-handling: 1.6.0-py36h37b9a7d_2    conda-forge
    geographiclib:          1.50-py_0               conda-forge
    geopy:                  1.22.0-pyh9f0ad1d_0     conda-forge
    krb5:                   1.16.3-hcfa6398_1001    conda-forge
    python_abi:             3.6-1_cp36m             conda-forge
    tqdm:                   4.46.0-pyh9f0ad1d_0     conda-forge

The following packages will be UPDATED:

    ca-certificates:        2018.03.07-0                        --> 2020.4.5.1-hecc5488_0     conda-forge
    certifi:                2018.4.16-py36_0                    --> 2020.4.5.1-py36h9f0ad1d_0 conda-forge
    conda:                  4.5.4-py36_0                        --> 4.8.3-py36h9f0ad1d_1      conda-forge
    cryptography:           2.2.2-py36h1de35cc_0                --> 2.

In [123]:
from bs4 import BeautifulSoup
import re
import seaborn as sns
from matplotlib import pyplot as plt

## Get Info from Wiki

In [124]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_Philadelphia_neighborhoods').text
soup = BeautifulSoup(source, 'lxml')

In [125]:
body = soup.find('ul')
tags = body('a')
ls = list()
a = 0
for tag in tags:
    a = a+1
    tag=str(tag)
    tag = tag.split('\">')[-1].split('<')[0].split('-')
    print(tag)

['Center City']
['South Philadelphia']
['Southwest Philadelphia']
['West Philadelphia']
['Lower North Philadelphia']
['Upper North Philadelphia']
['Bridesburg', 'Kensington', 'Port Richmond']
['Roxborough', 'Manayunk']
['Germantown', 'Chestnut Hill']
['Olney', 'Oak Lane']
['Near Northeast Philadelphia']
['Far Northeast Philadelphia']
['References']
['External links']


In [126]:
body = soup.find('ul')
tags = body('a')
ls = list()
a = 0
for tag in tags:
    a = a+1
    tag=str(tag)
    tag = tag.split('\">')[-1].split('<')[0].split('-')
    for name in tag:
        ls.append(name)
ls = ls[:-2]
ls.pop(5)
print(ls)

['Center City', 'South Philadelphia', 'Southwest Philadelphia', 'West Philadelphia', 'Lower North Philadelphia', 'Bridesburg', 'Kensington', 'Port Richmond', 'Roxborough', 'Manayunk', 'Germantown', 'Chestnut Hill', 'Olney', 'Oak Lane', 'Near Northeast Philadelphia', 'Far Northeast Philadelphia']


In [127]:
City_ls = list()
b = 0
while b<16:
    b = b+1
    City_ls.append('Philadelphia')
df = {'City': City_ls , 'Neighborhood': ls,}
df = pd.DataFrame(df)
df

,City,Neighborhood
0,Philadelphia,Center City
1,Philadelphia,South Philadelphia
2,Philadelphia,Southwest Philadelphia
3,Philadelphia,West Philadelphia
4,Philadelphia,Lower North Philadelphia
5,Philadelphia,Bridesburg
6,Philadelphia,Kensington
7,Philadelphia,Port Richmond
8,Philadelphia,Roxborough
9,Philadelphia,Manayunk


In [128]:
df.drop([4,14,15], inplace = True)

In [129]:
df['Latitude']=''
df['Longitude']=''
df

,City,Neighborhood,Latitude,Longitude
0,Philadelphia,Center City,,
1,Philadelphia,South Philadelphia,,
2,Philadelphia,Southwest Philadelphia,,
3,Philadelphia,West Philadelphia,,
5,Philadelphia,Bridesburg,,
6,Philadelphia,Kensington,,
7,Philadelphia,Port Richmond,,
8,Philadelphia,Roxborough,,
9,Philadelphia,Manayunk,,
10,Philadelphia,Germantown,,


In [130]:

N = df['Neighborhood'].tolist()
a = -1
for name in N:
    a = a+1
    address = name+', Philadelphia'
    geolocator = Nominatim(user_agent="Philadelphia_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    df['Latitude'][a] = latitude
    df['Longitude'][a] = longitude
    print(name, latitude, longitude)
    

Center City 39.9527237 -75.1635262
South Philadelphia 39.9125995 -75.16679028523015
Southwest Philadelphia 39.9527237 -75.1635262
West Philadelphia 39.9527237 -75.1635262
Bridesburg 40.0003901 -75.0696179
Kensington 39.9859458 -75.1318429
Port Richmond 39.9863549 -75.1009166
Roxborough 40.0432201 -75.229284
Manayunk 40.0245565 -75.2140682
Germantown 40.0434449 -75.1801775
Chestnut Hill 40.0770553 -75.2074
Olney 40.0412227 -75.1237863
Oak Lane 39.9527237 -75.1635262


In [133]:
df.to_csv(r'/Users/daiyiping/Desktop/01.csv', index = False)

In [134]:
df = pd.read_csv('/Users/daiyiping/Desktop/01.csv')

In [136]:
df.head(19)

,City,Neighborhood,Latitude,Longitude
0,Philadelphia,Center City,39.952724,-75.163526
1,Philadelphia,South Philadelphia,39.912599,-75.166790
2,Philadelphia,Southwest Philadelphia,39.952724,-75.163526
3,Philadelphia,West Philadelphia,39.952724,-75.163526
4,Philadelphia,Bridesburg,40.000390,-75.069618
5,Philadelphia,Kensington,39.985946,-75.131843
6,Philadelphia,Port Richmond,39.986355,-75.100917
7,Philadelphia,Roxborough,40.043220,-75.229284
8,Philadelphia,Manayunk,40.024557,-75.214068
9,Philadelphia,Germantown,40.043445,-75.180177


In [151]:
df.drop([2,3,12],inplace = True)

In [152]:
address = 'philadelphia, PA'

geolocator = Nominatim(user_agent="philadelphia_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of philadelphia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of philadelphia are 39.9527237, -75.1635262.


In [153]:
map_philadelphia = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_philadelphia)  
map_philadelphia

In [154]:
CLIENT_ID = '1FDF12VNFLTPBSVFTQTEJNMEZWKF0TPKKTLSHKSEE0UXE2K5' # your Foursquare ID
CLIENT_SECRET = 'BWLSQ3Z1TM35WRWL0PUBZ0ARPREFFF1PEBIRZIHRWKGQQD51' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: 1FDF12VNFLTPBSVFTQTEJNMEZWKF0TPKKTLSHKSEE0UXE2K5
CLIENT_SECRET:BWLSQ3Z1TM35WRWL0PUBZ0ARPREFFF1PEBIRZIHRWKGQQD51


In [155]:

neighborhood_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Center City are 39.9527237, -75.16352619999999.


In [156]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
#-->

#<!--
radius = 500 # define radius
#-->

#<!--
#\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?https://www.bostonmagazine.com/top-places-to-live-2018-single-family-homes/&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?https://www.bostonmagazine.com/top-places-to-live-2018-single-family-homes/&client_id=1FDF12VNFLTPBSVFTQTEJNMEZWKF0TPKKTLSHKSEE0UXE2K5&client_secret=BWLSQ3Z1TM35WRWL0PUBZ0ARPREFFF1PEBIRZIHRWKGQQD51&v=20180605&ll=39.9527237,-75.16352619999999&radius=500&limit=100'

In [157]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ebf4c061a4b0a00286bef19'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Avenue of the Arts South',
  'headerFullLocation': 'Avenue of the Arts South, Philadelphia',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 98,
  'suggestedBounds': {'ne': {'lat': 39.9572237045, 'lng': -75.15766688416947},
   'sw': {'lat': 39.9482236955, 'lng': -75.16938551583051}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bde0d566198c9b6c5cc12ff',
       'name': 'Dilworth Park',
       'location': {'address': '1 Penn Sq',
        'crossStreet': 'West Side of City Hall',
        'lat': 39.952771846343104,


In [158]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [159]:

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()


,name,categories,lat,lng
0,Dilworth Park,Park,39.952772,-75.164723
1,City Hall Courtyard,Plaza,39.952484,-75.163592
2,The Wanamaker Organ,Concert Hall,39.951536,-75.162752
3,"The Ritz-Carlton, Philadelphia",Hotel,39.951446,-75.164149
4,Fogo De Chão,Churrascaria,39.950917,-75.162971


In [160]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [226]:
philly_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Center City
South Philadelphia
Bridesburg
Kensington
Port Richmond
Roxborough
Manayunk
Germantown
Chestnut Hill
Olney


In [227]:
print(philly_venues.shape)
philly_venues.head()

(254, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Center City,39.952724,-75.163526,Dilworth Park,39.952772,-75.164723,Park
1,Center City,39.952724,-75.163526,City Hall Courtyard,39.952484,-75.163592,Plaza
2,Center City,39.952724,-75.163526,The Wanamaker Organ,39.951536,-75.162752,Concert Hall
3,Center City,39.952724,-75.163526,"The Ritz-Carlton, Philadelphia",39.951446,-75.164149,Hotel
4,Center City,39.952724,-75.163526,Fogo De Chão,39.950917,-75.162971,Churrascaria


In [228]:
philly_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bridesburg,12,12,12,12,12,12
Center City,98,98,98,98,98,98
Chestnut Hill,33,33,33,33,33,33
Germantown,12,12,12,12,12,12
Kensington,17,17,17,17,17,17
Manayunk,26,26,26,26,26,26
Olney,6,6,6,6,6,6
Port Richmond,16,16,16,16,16,16
Roxborough,20,20,20,20,20,20


In [229]:
philly_onehot = pd.get_dummies(philly_venues[['Venue Category']], prefix="", prefix_sep="")
philly_onehot['Neighborhood'] = philly_venues['Neighborhood'] 
fixed_columns = [philly_onehot.columns[-1]] + list(philly_onehot.columns[:-1])
philly_onehot = philly_onehot[fixed_columns]

philly_onehot.head()


,Neighborhood,Accessories Store,Advertising Agency,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Betting Shop,Big Box Store,Boutique,Breakfast Spot,Brewery,Burrito Place,Bus Station,Bus Stop,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Jewelry Store,Korean Restaurant,Light Rail Station,Liquor Store,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Museum,Music Venue,New American Restaurant,Optical Shop,Organic Grocery,Other Nightlife,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pub,Public Art,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Plaza,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Center City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Center City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Center City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Center City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Center City,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [230]:
philly_grouped = philly_onehot.groupby('Neighborhood').mean().reset_index()
philly_grouped.head()


,Neighborhood,Accessories Store,Advertising Agency,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Betting Shop,Big Box Store,Boutique,Breakfast Spot,Brewery,Burrito Place,Bus Station,Bus Stop,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Jewelry Store,Korean Restaurant,Light Rail Station,Liquor Store,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Museum,Music Venue,New American Restaurant,Optical Shop,Organic Grocery,Other Nightlife,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pub,Public Art,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Plaza,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Bridesburg,0.000000,0.0,0.083333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.083333,0.000000,0.083333,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.083333,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.083333,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.0,0.000000
1,Center City,0.000000,0.0,0.030612,0.0,0.000000,0.010204,0.020408,0.010204,0.000000,0.000000,0.010204,0.000000,0.020408,0.000000,0.0,0.0,0.000000,0.010204,0.0,0.010204,0.000000,0.000000,0.010204,0.0,0.000000,0.010204,0.010204,0.010204,0.010204,0.051020,0.010204,0.030612,0.0,0.010204,0.020408,0.0,0.0,0.000000,0.0,0.020408,0.010204,0.010204,0.0,0.000000,0.000000,0.010204,0.000000,0.0,0.0,0.0,0.010204,0.010204,0.010204,0.020408,0.010204,0.010204,0.000000,0.000000,0.061224,0.010204,0.020408,0.0,0.010204,0.000000,0.010204,0.0,0.000000,0.0,0.010204,0.0,0.020408,0.010204,0.000000,0.020408,0.000000,0.010204,0.000000,0.0,0.030612,0.010204,0.010204,0.000000,0.010204,0.000000,0.010204,0.000000,0.010204,0.000000,0.020408,0.020408,0.010204,0.000000,0.020408,0.020408,0.0,0.030612,0.010204,0.010204,0.010204,0.000000,0.0,0.020408,0.010204,0.030612,0.000000,0.010204,0.0,0.020408,0.0,0.0,0.010204,0.010204,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.010204,0.0,0.020408
2,Chestnut Hill,0.000000,0.0,0.121212,0.0,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.090909,0.030303,0.000000,0.0,0.0,0.030303,0.000000,0.0,0.000000,0.000000,0.000000,0.030303,0.0,0.030303,0.000000,0.000000,0.030303,0.000000,0.060606,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0

In [294]:
price = pd.read_csv('/Users/daiyiping/Desktop/philly_price.csv')
price.head(15)

,Neighborhood,Price
0,Bridesburg,187
1,Center City,393
2,Chestnut Hill,210
3,Germantown,192
4,Kensington,718
5,Manayunk,194
6,Olney,219
7,Port Richmond,176
8,Roxborough,188
9,South Philadelphia,236


In [232]:
philly_grouped = pd.merge(philly_grouped, price, on ='Neighborhood')
philly_grouped.head(10)


,Neighborhood,Accessories Store,Advertising Agency,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Betting Shop,Big Box Store,Boutique,Breakfast Spot,Brewery,Burrito Place,Bus Station,Bus Stop,Café,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doctor's Office,Donut Shop,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Historic Site,History Museum,Hotel,Ice Cream Shop,Indian Restaurant,Intersection,Israeli Restaurant,Italian Restaurant,Jewelry Store,Korean Restaurant,Light Rail Station,Liquor Store,Market,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Mobile Phone Shop,Movie Theater,Museum,Music Venue,New American Restaurant,Optical Shop,Organic Grocery,Other Nightlife,Park,Pet Store,Pharmacy,Photography Studio,Pizza Place,Playground,Plaza,Pub,Public Art,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Plaza,Smoke Shop,Snack Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Thai Restaurant,Thrift / Vintage Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio,Price
0,Bridesburg,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.083333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.0000,0.000000,187
1,Center City,0.000000,0.000000,0.030612,0.000000,0.000000,0.010204,0.020408,0.010204,0.000000,0.000000,0.010204,0.000000,0.020408,0.000000,0.000000,0.00,0.000000,0.010204,0.000000,0.010204,0.000000,0.000000,0.010204,0.000000,0.000000,0.010204,0.010204,0.010204,0.010204,0.051020,0.010204,0.030612,0.000000,0.010204,0.020408,0.00,0.00,0.000000,0.00,0.020408,0.010204,0.010204,0.00,0.000000,0.000000,0.010204,0.000000,0.00,0.000000,0.000000,0.010204,0.010204,0.010204,0.020408,0.010204,0.010204,0.000000,0.000000,0.061224,0.010204,0.020408,0.000000,0.010204,0.000000,0.010204,0.000000,0.000000,0.00,0.010204,0.000000,0.020408,0.010204,0.000000,0.020408,0.000000,0.010204,0.000000,0.0000,0.030612,0.010204,0.010204,0.000000,0.010204,0.000000,0.010204,0.000000,0.010204,0.000000,0.020408,0.020408,0.010204,0.000000,0.020408,0.020408,0.0000,0.030612,0.010204,0.010204,0.010204,0.000000,0.00,0.020408,0.010204,0.030612,0.000000,0.010204,0.000000,0.020408,0.000000,0.00,0.010204,0.010204,0.000000,0.000000,0.000000,0.020408,0.000000,0.000000,0.010204,0.0000,0.020408,393
2,Chestnut Hill,0.000000,0.000000,0.121212,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.090909,0.030303,0.000000,0.000000,0.00,0.030303,

In [233]:
from sklearn.preprocessing import StandardScaler
X = philly_grouped.values[:,1:]
X = np.nan_to_num(X)
Clus_dataSet = StandardScaler().fit_transform(X)
Clus_dataSet

/Users/daiyiping/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/daiyiping/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-0.33333333, -0.33333333,  1.44891439, ..., -0.33333333,
        -0.33333333, -0.52535643],
       [-0.33333333, -0.33333333,  0.1718924 , ..., -0.33333333,
         3.        ,  0.75843271],
       [-0.33333333, -0.33333333,  2.36642286, ..., -0.33333333,
        -0.33333333, -0.38202075],
       ...,
       [-0.33333333, -0.33333333, -0.56960424, ...,  3.        ,
        -0.33333333, -0.59390828],
       [-0.33333333, -0.33333333, -0.56960424, ..., -0.33333333,
        -0.33333333, -0.51912445],
       [-0.33333333, -0.33333333, -0.56960424, ..., -0.33333333,
        -0.33333333, -0.21998911]])

In [283]:
kclusters = 4
k_means = KMeans(init = "k-means++", n_clusters = kclusters, n_init = 12)
k_means.fit(X)
labels = k_means.labels_
print(labels)

[0 2 3 0 1 0 3 0 0 3]


In [284]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [285]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = philly_grouped['Neighborhood']

for ind in np.arange(philly_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(philly_grouped.iloc[ind, :-1], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bridesburg,Pizza Place,Café,Food & Drink Shop,Convenience Store,Speakeasy,Baseball Field,Bar,Bus Station,Gym / Fitness Center,Video Store
1,Center City,Hotel,Coffee Shop,American Restaurant,Spa,Convenience Store,Sandwich Place,New American Restaurant,Yoga Studio,Mexican Restaurant,Restaurant
2,Chestnut Hill,American Restaurant,Bank,Coffee Shop,Pizza Place,Playground,Cheese Shop,Market,Sandwich Place,Café,Record Shop
3,Germantown,Vegetarian / Vegan Restaurant,Accessories Store,BBQ Joint,Museum,Pharmacy,Photography Studio,History Museum,Bus Stop,Historic Site,Art Gallery
4,Kensington,Vietnamese Restaurant,Chinese Restaurant,Athletics & Sports,Metro Station,Light Rail Station,Pizza Place,Other Nightlife,Train Station,Thrift / Vintage Store,Mobile Phone Shop


In [286]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', labels)

philly_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
philly_merged = philly_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

philly_merged.head() # check the last columns!

,City,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Philadelphia,Center City,39.952724,-75.163526,2,Hotel,Coffee Shop,American Restaurant,Spa,Convenience Store,Sandwich Place,New American Restaurant,Yoga Studio,Mexican Restaurant,Restaurant
1,Philadelphia,South Philadelphia,39.912599,-75.166790,3,Intersection,Restaurant,Breakfast Spot,Bar,Pizza Place,Cosmetics Shop,Men's Store,Pharmacy,Seafood Restaurant,Betting Shop
4,Philadelphia,Bridesburg,40.000390,-75.069618,0,Pizza Place,Café,Food & Drink Shop,Convenience Store,Speakeasy,Baseball Field,Bar,Bus Station,Gym / Fitness Center,Video Store
5,Philadelphia,Kensington,39.985946,-75.131843,1,Vietnamese Restaurant,Chinese Restaurant,Athletics & Sports,Metro Station,Light Rail Station,Pizza Place,Other Nightlife,Train Station,Thrift / Vintage Store,Mobile Phone Shop
6,Philadelphia,Port Richmond,39.986355,-75.100917,0,Bar,Coffee Shop,Bakery,Pet Store,Park,Department Store,Salon / Barbershop,Music Venue,Snack Place,Pizza Place


In [287]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(philly_merged['Latitude'], philly_merged['Longitude'], philly_merged['Neighborhood'], philly_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


In [288]:
cluster0 = philly_merged.loc[philly_merged['Cluster Labels'] == 0, philly_merged.columns[[1] + list(range(5, philly_merged.shape[1]))]]
cluster2 = philly_merged.loc[philly_merged['Cluster Labels'] == 2, philly_merged.columns[[1] + list(range(5, philly_merged.shape[1]))]]
cluster1 = philly_merged.loc[philly_merged['Cluster Labels'] == 1, philly_merged.columns[[1] + list(range(5, philly_merged.shape[1]))]]
cluster3 = philly_merged.loc[philly_merged['Cluster Labels'] == 3, philly_merged.columns[[1] + list(range(5, philly_merged.shape[1]))]]


In [289]:
cluster0_with_price = pd.merge(cluster0, price, on = 'Neighborhood')
cluster1_with_price = pd.merge(cluster1, price, on = 'Neighborhood')
cluster2_with_price = pd.merge(cluster2, price, on = 'Neighborhood')
cluster3_with_price = pd.merge(cluster3, price, on = 'Neighborhood')

In [290]:
cluster0_with_price

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Price
0,Bridesburg,Pizza Place,Café,Food & Drink Shop,Convenience Store,Speakeasy,Baseball Field,Bar,Bus Station,Gym / Fitness Center,Video Store,187
1,Port Richmond,Bar,Coffee Shop,Bakery,Pet Store,Park,Department Store,Salon / Barbershop,Music Venue,Snack Place,Pizza Place,176
2,Roxborough,Coffee Shop,Bank,Dessert Shop,Diner,Discount Store,Chinese Restaurant,Doctor's Office,Donut Shop,Mobile Phone Shop,Fast Food Restaurant,188
3,Manayunk,Pizza Place,Bar,Furniture / Home Store,Food Truck,Coffee Shop,Dessert Shop,Dive Bar,Caribbean Restaurant,Brewery,Food & Drink Shop,194
4,Germantown,Vegetarian / Vegan Restaurant,Accessories Store,BBQ Joint,Museum,Pharmacy,Photography Studio,History Museum,Bus Stop,Historic Site,Art Gallery,192


In [291]:
cluster1_with_price

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Price
0,Kensington,Vietnamese Restaurant,Chinese Restaurant,Athletics & Sports,Metro Station,Light Rail Station,Pizza Place,Other Nightlife,Train Station,Thrift / Vintage Store,Mobile Phone Shop,718


In [292]:
cluster2_with_price

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Price
0,Center City,Hotel,Coffee Shop,American Restaurant,Spa,Convenience Store,Sandwich Place,New American Restaurant,Yoga Studio,Mexican Restaurant,Restaurant,393


In [293]:
cluster3_with_price

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Price
0,South Philadelphia,Intersection,Restaurant,Breakfast Spot,Bar,Pizza Place,Cosmetics Shop,Men's Store,Pharmacy,Seafood Restaurant,Betting Shop,236
1,Chestnut Hill,American Restaurant,Bank,Coffee Shop,Pizza Place,Playground,Cheese Shop,Market,Sandwich Place,Café,Record Shop,210
2,Olney,Korean Restaurant,Convenience Store,Pizza Place,Park,Strip Club,Fast Food Restaurant,Cosmetics Shop,Department Store,Dessert Shop,Diner,219


In [298]:
cluster0_with_price.mean()

Price    187.4
dtype: float64

In [299]:
cluster1_with_price.mean()

Price    718.0
dtype: float64

In [300]:
cluster2_with_price.mean()

Price    393.0
dtype: float64

In [301]:
cluster3_with_price.mean()

Price    221.666667
dtype: float64